# 兰州天气数据可视化分析（适用于数字大屏）

使用 pyecharts 创建交互式可视化图表，为后期数字大屏展示做准备。

In [68]:
# 导入必要的库
import pandas as pd
import numpy as np
from pyecharts import options as opts
from pyecharts.charts import Line, Bar, Pie, Calendar, Grid, Scatter, Radar, Page
from pyecharts.globals import ThemeType
import re
from datetime import datetime

In [3]:
# 数据预处理函数
def extract_temperature(temp_str):
    return int(temp_str.replace('℃', ''))

def extract_wind_direction(wind_str):
    if '无持续风向' in wind_str:
        return '无持续风向'
    return wind_str.split()[0]

def extract_wind_force(wind_str):
    match = re.search(r'(\d+)-(\d+)级', wind_str)
    if match:
        return (int(match.group(1)) + int(match.group(2))) / 2
    return np.nan

# 读取和处理数据
df = pd.read_excel('../数据集/lanzhou_weather_data.xlsx')

# 处理日期
df['日期'] = pd.to_datetime(df['日期'], format='%Y年%m月%d日')
df['年'] = df['日期'].dt.year
df['月'] = df['日期'].dt.month
df['季节'] = pd.cut(df['月'], bins=[0,3,6,9,12], labels=['春季','夏季','秋季','冬季'])

# 处理温度
df['最高气温_数值'] = df['最高气温'].apply(extract_temperature)
df['最低气温_数值'] = df['最低气温'].apply(extract_temperature)
df['日温差'] = df['最高气温_数值'] - df['最低气温_数值']

# 处理风向和风力
df['白天风向'] = df['白天风力'].apply(extract_wind_direction)
df['夜间风向'] = df['夜间风力'].apply(extract_wind_direction)
df['白天风力等级'] = df['白天风力'].apply(extract_wind_force)
df['夜间风力等级'] = df['夜间风力'].apply(extract_wind_force)


# 添加温度日历图所需的数据处理
df['temperature_str'] = df.apply(
    lambda x: f"最高: {x['最高气温_数值']}℃\n最低: {x['最低气温_数值']}℃",
    axis=1
)

## 1. 温度变化趋势（折线图）

In [5]:
def create_temperature_line():
    monthly_temp = df.groupby(['年', '月']).agg({
        '最高气温_数值': 'mean',
        '最低气温_数值': 'mean'
    }).reset_index()
    
    line = (
        Line(init_opts=opts.InitOpts(theme=ThemeType.MACARONS, width='1200px', height='600px'))
        .add_xaxis([f"{int(year)}年{int(month)}月" for year, month in zip(monthly_temp['年'], monthly_temp['月'])])
        .add_yaxis(
            "月平均最高气温", 
            monthly_temp['最高气温_数值'].round(1).tolist(),
            markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_="max")]),
            linestyle_opts=opts.LineStyleOpts(width=2)
        )
        .add_yaxis(
            "月平均最低气温", 
            monthly_temp['最低气温_数值'].round(1).tolist(),
            markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_="min")]),
            linestyle_opts=opts.LineStyleOpts(width=2)
        )
        .set_global_opts(
            title_opts=opts.TitleOpts(title="兰州市月度气温变化趋势"),
            tooltip_opts=opts.TooltipOpts(trigger="axis", axis_pointer_type="cross"),
            toolbox_opts=opts.ToolboxOpts(feature={"saveAsImage": {}}),
            xaxis_opts=opts.AxisOpts(type_="category", boundary_gap=False),
            yaxis_opts=opts.AxisOpts(type_="value", name="温度(℃)"),
            datazoom_opts=[opts.DataZoomOpts(), opts.DataZoomOpts(type_="inside")]
        )
    )
    line.render('../数据分析/可视化图表/温度变化趋势图.html')

create_temperature_line()

## 2. 天气类型分布（饼图）

In [7]:
def create_weather_pie():
    weather_counts = df['白天天气'].value_counts()
    
    pie = (
        Pie(init_opts=opts.InitOpts(theme=ThemeType.WESTEROS, width='1200px', height='600px'))
        .add(
            series_name="天气类型分布",
            data_pair=[(k, v) for k, v in weather_counts.items()],
            radius=["30%", "75%"],
            rosetype="radius"
        )
        .set_global_opts(
            title_opts=opts.TitleOpts(title="兰州市天气类型分布"),
            legend_opts=opts.LegendOpts(orient="vertical", pos_left="85%", pos_top="middle"),
            toolbox_opts=opts.ToolboxOpts(feature={"saveAsImage": {}})
        )
        .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {c}"))
    )
    pie.render('../数据分析/可视化图表/天气类型分布图.html')

create_weather_pie()

## 3. 季节性天气分布（堆叠柱状图）

In [9]:
def create_seasonal_weather_bar():
    seasonal_weather = pd.crosstab(df['季节'], df['白天天气'])
    
    bar = (
        Bar(init_opts=opts.InitOpts(theme=ThemeType.LIGHT, width='1200px', height='600px'))
        .add_xaxis(seasonal_weather.index.tolist())
    )
    
    for weather_type in seasonal_weather.columns:
        bar.add_yaxis(
            weather_type,
            seasonal_weather[weather_type].tolist(),
            stack="stack1",
            label_opts=opts.LabelOpts(is_show=False)
        )
    
    bar.set_global_opts(
        title_opts=opts.TitleOpts(title="季节性天气分布"),
        legend_opts=opts.LegendOpts(pos_right="15%"),
        toolbox_opts=opts.ToolboxOpts(feature={"saveAsImage": {}}),
        xaxis_opts=opts.AxisOpts(type_="category"),
        yaxis_opts=opts.AxisOpts(type_="value", name="频次"),
        datazoom_opts=[opts.DataZoomOpts(), opts.DataZoomOpts(type_="inside")]
    )
    
    bar.render('../数据分析/可视化图表/季节性天气分布图.html')

create_seasonal_weather_bar()

## 4. 风向分布（雷达图）

In [11]:
def create_wind_radar():
    wind_dir_counts = df['白天风向'].value_counts()
    
    radar = (
        Radar(init_opts=opts.InitOpts(theme=ThemeType.CHALK, width='1200px', height='600px'))
        .add_schema(
            schema=[
                opts.RadarIndicatorItem(name=k, max_=wind_dir_counts.max())
                for k in wind_dir_counts.index
            ]
        )
        .add(
            "风向分布",
            [wind_dir_counts.tolist()],
            areastyle_opts=opts.AreaStyleOpts(opacity=0.3)
        )
        .set_global_opts(
            title_opts=opts.TitleOpts(title="风向分布雷达图"),
            toolbox_opts=opts.ToolboxOpts(feature={"saveAsImage": {}})
        )
    )
    radar.render('../数据分析/可视化图表/风向分布雷达图.html')

create_wind_radar()

## 5. 温度关系（散点图）

In [13]:
def create_temperature_scatter():
    scatter = (
        Scatter(init_opts=opts.InitOpts(theme=ThemeType.ROMANTIC, width='1200px', height='600px'))
        .add_xaxis(df['最高气温_数值'].tolist())
        .add_yaxis(
            "温度关系",
            df['最低气温_数值'].tolist(),
            label_opts=opts.LabelOpts(is_show=False)
        )
        .set_global_opts(
            title_opts=opts.TitleOpts(title="最高温度与最低温度关系散点图"),
            xaxis_opts=opts.AxisOpts(type_="value", name="最高气温(℃)"),
            yaxis_opts=opts.AxisOpts(type_="value", name="最低气温(℃)"),
            visualmap_opts=opts.VisualMapOpts(type_="size", max_=40, min_=-10),
            toolbox_opts=opts.ToolboxOpts(feature={"saveAsImage": {}})
        )
    )
    scatter.render('../数据分析/可视化图表/温度关系散点图.html')

create_temperature_scatter()

## 6. 2024温度日历图

In [50]:
def create_calendar_2024():
    """创建2024年温度日历图"""
    # 准备2024年数据
    year_data = df[df['年'] == 2024].copy()
    data = [
        [d.strftime("%Y-%m-%d"), float(t)]
        for d, t in zip(year_data['日期'], year_data['最高气温_数值'])
    ]
    
    # 创建日历图
    calendar = (
        Calendar()
        .add(
            "",  # 不显示系列名
            data,
            calendar_opts=opts.CalendarOpts(
                range_="2024",
                daylabel_opts=opts.CalendarDayLabelOpts(name_map="cn"),
                monthlabel_opts=opts.CalendarMonthLabelOpts(name_map="cn")
            )
        )
        .set_global_opts(
            title_opts=opts.TitleOpts(title="2024年兰州市气温日历图"),
            visualmap_opts=opts.VisualMapOpts(
                min_=0, max_=40,
                pieces=[
                    {"min": 0, "max": 10, "color": "#87CEEB"},
                    {"min": 10, "max": 20, "color": "#90EE90"},
                    {"min": 20, "max": 30, "color": "#FFD700"},
                    {"min": 30, "max": 40, "color": "#FF4500"}
                ]
            )
        )
    )
    
    # 生成图表
    calendar.render('可视化图表/2024年温度日历图.html')

# 生成2024年温度日历图
create_calendar_2024()

## 7. 综合仪表盘

In [92]:
# def create_dashboard():
#     # 计算月度温度数据
#     monthly_temp = df.groupby(['年', '月']).agg({
#         '最高气温_数值': 'mean',
#         '最低气温_数值': 'mean'
#     }).reset_index()
    
#     # 创建所有图表（不要调用render方法）
#     temp_line = (
#         Line(init_opts=opts.InitOpts(theme=ThemeType.MACARONS, width='1200px', height='600px'))
#         .add_xaxis([f"{int(year)}年{int(month)}月" for year, month in zip(monthly_temp['年'], monthly_temp['月'])])
#         .add_yaxis(
#             "月平均最高气温", 
#             monthly_temp['最高气温_数值'].round(1).tolist(),
#             markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_="max")]),
#             linestyle_opts=opts.LineStyleOpts(width=2)
#         )
#         .add_yaxis(
#             "月平均最低气温", 
#             monthly_temp['最低气温_数值'].round(1).tolist(),
#             markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_="min")]),
#             linestyle_opts=opts.LineStyleOpts(width=2)
#         )
#         .set_global_opts(
#             title_opts=opts.TitleOpts(title="兰州市月度气温变化趋势"),
#             tooltip_opts=opts.TooltipOpts(trigger="axis", axis_pointer_type="cross"),
#             xaxis_opts=opts.AxisOpts(type_="category", boundary_gap=False),
#             yaxis_opts=opts.AxisOpts(type_="value", name="温度(℃)")
#         )
#     )
    
#     weather_counts = df['白天天气'].value_counts()
#     weather_pie = (
#         Pie(init_opts=opts.InitOpts(theme=ThemeType.WESTEROS, width='1200px', height='600px'))
#         .add(
#             series_name="天气类型分布",
#             data_pair=[(k, v) for k, v in weather_counts.items()],
#             radius=["30%", "75%"],
#             rosetype="radius"
#         )
#         .set_global_opts(
#             title_opts=opts.TitleOpts(title="兰州市天气类型分布"),
#             legend_opts=opts.LegendOpts(orient="vertical", pos_left="85%", pos_top="middle")
#         )
#         .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {c}"))
#     )
    
#     seasonal_weather = pd.crosstab(df['季节'], df['白天天气'])
#     seasonal_bar = (
#         Bar(init_opts=opts.InitOpts(theme=ThemeType.LIGHT, width='1200px', height='600px'))
#         .add_xaxis(seasonal_weather.index.tolist())
#     )
    
#     for weather_type in seasonal_weather.columns:
#         seasonal_bar.add_yaxis(
#             weather_type,
#             seasonal_weather[weather_type].tolist(),
#             stack="stack1",
#             label_opts=opts.LabelOpts(is_show=False)
#         )
    
#     seasonal_bar.set_global_opts(
#         title_opts=opts.TitleOpts(title="季节性天气分布"),
#         legend_opts=opts.LegendOpts(pos_right="15%"),
#         xaxis_opts=opts.AxisOpts(type_="category"),
#         yaxis_opts=opts.AxisOpts(type_="value", name="频次")
#     )
    
#     wind_dir_counts = df['白天风向'].value_counts()
#     wind_radar = (
#         Radar(init_opts=opts.InitOpts(theme=ThemeType.CHALK, width='1200px', height='600px'))
#         .add_schema(
#             schema=[
#                 opts.RadarIndicatorItem(name=k, max_=wind_dir_counts.max())
#                 for k in wind_dir_counts.index
#             ]
#         )
#         .add(
#             "风向分布",
#             [wind_dir_counts.tolist()],
#             areastyle_opts=opts.AreaStyleOpts(opacity=0.3)
#         )
#         .set_global_opts(
#             title_opts=opts.TitleOpts(title="风向分布雷达图")
#         )
#     )
    
#     temp_scatter = (
#         Scatter(init_opts=opts.InitOpts(theme=ThemeType.ROMANTIC, width='1200px', height='600px'))
#         .add_xaxis(df['最高气温_数值'].tolist())
#         .add_yaxis(
#             "温度关系",
#             df['最低气温_数值'].tolist(),
#             label_opts=opts.LabelOpts(is_show=False)
#         )
#         .set_global_opts(
#             title_opts=opts.TitleOpts(title="最高温度与最低温度关系散点图"),
#             xaxis_opts=opts.AxisOpts(type_="value", name="最高气温(℃)"),
#             yaxis_opts=opts.AxisOpts(type_="value", name="最低气温(℃)"),
#             visualmap_opts=opts.VisualMapOpts(type_="size", max_=40, min_=-10)
#         )
#     )
    
#     # 添加2024年温度日历图
#     year_data = df[df['年'] == 2024].copy()
#     data = [
#         [d.strftime("%Y-%m-%d"), float(t)]
#         for d, t in zip(year_data['日期'], year_data['最高气温_数值'])
#     ]
    
#     calendar = (
#         Calendar(init_opts=opts.InitOpts(theme=ThemeType.WESTEROS, width='1200px', height='600px'))
#         .add(
#             "",  # 不显示系列名
#             data,
#             calendar_opts=opts.CalendarOpts(
#                 range_="2024",
#                 daylabel_opts=opts.CalendarDayLabelOpts(name_map="cn"),
#                 monthlabel_opts=opts.CalendarMonthLabelOpts(name_map="cn")
#             )
#         )
#         .set_global_opts(
#             title_opts=opts.TitleOpts(title="2024年兰州市气温日历图"),
#             visualmap_opts=opts.VisualMapOpts(
#                 min_=0, max_=40,
#                 pieces=[
#                     {"min": 0, "max": 10, "color": "#87CEEB"},
#                     {"min": 10, "max": 20, "color": "#90EE90"},
#                     {"min": 20, "max": 30, "color": "#FFD700"},
#                     {"min": 30, "max": 40, "color": "#FF4500"}
#                 ]
#             )
#         )
#     )
    
#     # 创建页面对象
#     page = Page(layout=Page.SimplePageLayout)
    
#     # 添加所有图表
#     page.add(
#         temp_line,
#         weather_pie,
#         seasonal_bar,
#         wind_radar,
#         temp_scatter,
#         calendar  # 添加日历图
#     )
    
#     # 生成HTML文件
#     page.render('兰州天气综合仪表盘.html')

# # 执行仪表盘创建
# create_dashboard()

In [102]:
from pyecharts.charts import Page
from pyecharts.globals import ThemeType
from pyecharts.commons.utils import JsCode

def create_dashboard():
    # 设置全局初始化选项
    init_opts = opts.InitOpts(
        width="1200px",
        height="600px",
        theme=ThemeType.MACARONS,
        page_title="兰州天气数据可视化分析",
        js_host="https://assets.pyecharts.org/assets/"
    )
    
    # 计算月度温度数据
    monthly_temp = df.groupby(['年', '月']).agg({
        '最高气温_数值': 'mean',
        '最低气温_数值': 'mean'
    }).reset_index()
    
    # 温度趋势图
    temp_line = (
        Line(init_opts=init_opts)
        .add_xaxis([f"{int(year)}年{int(month)}月" for year, month in zip(monthly_temp['年'], monthly_temp['月'])])
        .add_yaxis(
            "月平均最高气温", 
            monthly_temp['最高气温_数值'].round(1).tolist(),
            markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_="max")]),
            linestyle_opts=opts.LineStyleOpts(width=2),
            itemstyle_opts=opts.ItemStyleOpts(
                color="#FF6B6B",
                border_width=2,
            )
        )
        .add_yaxis(
            "月平均最低气温", 
            monthly_temp['最低气温_数值'].round(1).tolist(),
            markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_="min")]),
            linestyle_opts=opts.LineStyleOpts(width=2),
            itemstyle_opts=opts.ItemStyleOpts(
                color="#4ECDC4",
                border_width=2,
            )
        )
        .set_global_opts(
            title_opts=opts.TitleOpts(
                title="兰州市月度气温变化趋势",
                subtitle="2024年温度变化情况",
                pos_left="center"
            ),
            tooltip_opts=opts.TooltipOpts(
                trigger="axis",
                axis_pointer_type="cross",
                background_color="rgba(255,255,255,0.8)",
                border_color="#CCC",
                border_width=1,
                textstyle_opts=opts.TextStyleOpts(color="#000")
            ),
            xaxis_opts=opts.AxisOpts(
                type_="category",
                boundary_gap=False,
                axislabel_opts=opts.LabelOpts(rotate=45)
            ),
            yaxis_opts=opts.AxisOpts(
                type_="value",
                name="温度(℃)",
                name_location="middle",
                name_gap=40
            ),
            datazoom_opts=[
                opts.DataZoomOpts(range_start=0, range_end=100),
                opts.DataZoomOpts(type_="inside")
            ],
            legend_opts=opts.LegendOpts(pos_top="5%")
        )
    )
    
    # 天气类型饼图
    weather_counts = df['白天天气'].value_counts()
    color_list = ['#FF9999', '#66B2FF', '#99FF99', '#FFCC99', '#FF99CC', '#99CCFF', '#FFB366', '#99FF99']
    weather_pie = (
        Pie(init_opts=init_opts)
        .add(
            series_name="天气类型分布",
            data_pair=[(k, v) for k, v in weather_counts.items()],
            radius=["35%", "70%"],
            center=["50%", "50%"],
            rosetype="radius",
            itemstyle_opts=opts.ItemStyleOpts(
                border_color="#fff",
                border_width=2
            )
        )
        .set_colors(color_list)
        .set_global_opts(
            title_opts=opts.TitleOpts(
                title="兰州市天气类型分布",
                subtitle="各类天气占比情况",
                pos_left="center"
            ),
            legend_opts=opts.LegendOpts(
                orient="vertical",
                pos_left="85%",
                pos_top="middle",
                item_gap=12
            )
        )
        .set_series_opts(
            label_opts=opts.LabelOpts(
                formatter="{b}: {c}次\n{d}%",
                font_size=12,
                font_weight="bold"
            )
        )
    )
    
    # 季节性天气分布图
    seasonal_weather = pd.crosstab(df['季节'], df['白天天气'])
    seasonal_bar = (
        Bar(init_opts=init_opts)
        .add_xaxis(seasonal_weather.index.tolist())
    )
    
    for idx, weather_type in enumerate(seasonal_weather.columns):
        seasonal_bar.add_yaxis(
            weather_type,
            seasonal_weather[weather_type].tolist(),
            stack="stack1",
            label_opts=opts.LabelOpts(is_show=False),
            itemstyle_opts=opts.ItemStyleOpts(
                color=color_list[idx % len(color_list)],
                opacity=0.8
            )
        )
    
    seasonal_bar.set_global_opts(
        title_opts=opts.TitleOpts(
            title="季节性天气分布",
            subtitle="各季节天气类型统计",
            pos_left="center"
        ),
        legend_opts=opts.LegendOpts(
            pos_right="5%",
            orient="vertical",
            pos_top="15%"
        ),
        xaxis_opts=opts.AxisOpts(
            type_="category",
            axislabel_opts=opts.LabelOpts(font_size=12)
        ),
        yaxis_opts=opts.AxisOpts(
            type_="value",
            name="频次",
            name_location="middle",
            name_gap=40
        ),
        tooltip_opts=opts.TooltipOpts(
            trigger="axis",
            axis_pointer_type="shadow"
        )
    )
    

    # 风向雷达图
    wind_dir_counts = df['白天风向'].value_counts()
    wind_radar = (
        Radar(init_opts=init_opts)
        .add_schema(
            schema=[
                opts.RadarIndicatorItem(name=k, max_=wind_dir_counts.max())
                for k in wind_dir_counts.index
            ],
            splitarea_opt=opts.SplitAreaOpts(
                is_show=True,
                areastyle_opts=opts.AreaStyleOpts(opacity=1)
            ),
            textstyle_opts=opts.TextStyleOpts(color="#000")
        )
        .add(
            "风向分布",
            [wind_dir_counts.tolist()],
            areastyle_opts=opts.AreaStyleOpts(opacity=0.8),
            linestyle_opts=opts.LineStyleOpts(
                width=2,
                color="#91CC75"
            )
        )
        .set_global_opts(
            title_opts=opts.TitleOpts(
                title="风向分布雷达图",
                subtitle="各方位风向频次",
                pos_left="center"
            )
        )
    )

    
    # 温度散点图
    temp_scatter = (
        Scatter(init_opts=init_opts)
        .add_xaxis(df['最高气温_数值'].tolist())
        .add_yaxis(
            "温度关系",
            df['最低气温_数值'].tolist(),
            label_opts=opts.LabelOpts(is_show=False),
            itemstyle_opts=opts.ItemStyleOpts(
                color="#5470C6",
                opacity=0.6,
                border_color="#fff",
                border_width=1
            )
        )
        .set_global_opts(
            title_opts=opts.TitleOpts(
                title="最高温度与最低温度关系散点图",
                subtitle="温度相关性分析",
                pos_left="center"
            ),
            xaxis_opts=opts.AxisOpts(
                type_="value",
                name="最高气温(℃)",
                name_location="middle",
                name_gap=30,
                splitline_opts=opts.SplitLineOpts(is_show=True)
            ),
            yaxis_opts=opts.AxisOpts(
                type_="value",
                name="最低气温(℃)",
                name_location="middle",
                name_gap=30,
                splitline_opts=opts.SplitLineOpts(is_show=True)
            ),
            visualmap_opts=opts.VisualMapOpts(
                type_="size",
                max_=40,
                min_=-10,
                pos_left="10%"
            )
        )
    )
    
    # 温度日历图
    year_data = df[df['年'] == 2024].copy()
    data = [
        [d.strftime("%Y-%m-%d"), float(t)]
        for d, t in zip(year_data['日期'], year_data['最高气温_数值'])
    ]
    
    calendar = (
        Calendar(init_opts=init_opts)
        .add(
            "",
            data,
            calendar_opts=opts.CalendarOpts(
                range_="2024",
                daylabel_opts=opts.CalendarDayLabelOpts(name_map="cn"),
                monthlabel_opts=opts.CalendarMonthLabelOpts(name_map="cn"),
                pos_top="120",
                pos_left="30",
                pos_right="30"
            )
        )
        .set_global_opts(
            title_opts=opts.TitleOpts(
                title="2024年兰州市气温日历图",
                subtitle="每日最高温度分布",
                pos_left="center"
            ),
            visualmap_opts=opts.VisualMapOpts(
                min_=0,
                max_=40,
                orient="horizontal",
                is_piecewise=True,
                pos_bottom="0%",
                pieces=[
                    {"min": 0, "max": 10, "label": "0-10℃", "color": "#87CEEB"},
                    {"min": 10, "max": 20, "label": "10-20℃", "color": "#90EE90"},
                    {"min": 20, "max": 30, "label": "20-30℃", "color": "#FFD700"},
                    {"min": 30, "max": 40, "label": "30-40℃", "color": "#FF4500"}
                ]
            )
        )
    )
    
    # 创建页面对象，使用更紧凑的布局
    page = Page(layout=Page.DraggablePageLayout)
    
    # 添加所有图表
    page.add(
        temp_line,
        weather_pie,
        seasonal_bar,
        wind_radar,
        temp_scatter,
        calendar
    )
    
    # 生成HTML文件，添加更多配置
    page.render(
        path='可视化图表/兰州天气综合仪表盘.html',
        template_name='simple_page.html',
        chart_config={
            "renderer": "canvas",
            "width": "100%",
            "height": "100%"
        }
    )

# 执行仪表盘创建
create_dashboard()

## 后续数字大屏开发建议

1. 布局设计：
   - 使用 Grid 布局系统
   - 合理安排各图表位置
   - 添加实时数据更新功能

2. 交互设计：
   - 添加时间选择器
   - 增加图表联动效果
   - 添加数据筛选功能

3. 样式优化：
   - 统一配色方案
   - 优化字体和图表大小
   - 添加动画效果

4. 功能扩展：
   - 添加天气预警模块
   - 集成地图展示
   - 添加数据对比功能